In [1]:
!pip install pyspark

In [2]:
import pandas as pd

In [3]:
from google.colab import userdata
AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')

In [4]:
BUCKET = "steam-reco-team-yw1204"
DT = "dt=1205"
RAW_BASE = f"s3a://{BUCKET}/raw/steam_kaggle/{DT}"

print(RAW_BASE)

s3a://steam-reco-team-yw1204/raw/steam_kaggle/dt=1205


In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("SteamReco-Colab")
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
    .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID)
    .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("spark.sql.shuffle.partitions", "64")
    .getOrCreate()
)

spark

In [6]:
reviews = spark.read.parquet("s3a://steam-reco-team-yw1204/curated/reviews_clean/v=1")
games   = spark.read.parquet("s3a://steam-reco-team-yw1204/curated/games_clean/v=1")
users   = spark.read.parquet("s3a://steam-reco-team-yw1204/curated/users_clean/v=1")

reviews.printSchema()
games.printSchema()
users.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- is_recommended: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- funny_votes: integer (nullable = true)
 |-- playtime_hours: double (nullable = true)
 |-- review_date: date (nullable = true)
 |-- review_month: integer (nullable = true)
 |-- review_year: integer (nullable = true)

root
 |-- app_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- date_release: string (nullable = true)
 |-- win: boolean (nullable = true)
 |-- mac: boolean (nullable = true)
 |-- linux: boolean (nullable = true)
 |-- rating: string (nullable = true)
 |-- positive_ratio: integer (nullable = true)
 |-- user_reviews: integer (nullable = true)
 |-- price_final: double (nullable = true)
 |-- price_original: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- steam_deck: boolean (nullab

# Content Based Filtering

In [7]:
from pyspark.sql import functions as F

games_clean2 = (
    games
    .withColumn("description_clean", F.lower(F.col("description")))
    .withColumn("tags_clean", F.col("tags"))
    .na.fill({"description_clean": ""})
)


In [8]:
from pyspark.ml.feature import RegexTokenizer, HashingTF, IDF, CountVectorizer
from pyspark.ml import Pipeline

tokenizer = RegexTokenizer(inputCol="description_clean", outputCol="desc_words", pattern="\\W+")
hashTF = HashingTF(inputCol="desc_words", outputCol="desc_tf", numFeatures=4096)
idf = IDF(inputCol="desc_tf", outputCol="desc_tfidf")
tag_cv = CountVectorizer(inputCol="tags_clean", outputCol="tags_vec")

pipeline = Pipeline(stages=[tokenizer, hashTF, idf, tag_cv])
model = pipeline.fit(games_clean2)
games_vec = model.transform(games_clean2)


In [9]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["desc_tfidf", "tags_vec"],
    outputCol="content_vec"
)

games_final = assembler.transform(games_vec).select("app_id", "title", "content_vec")
games_final.cache()


DataFrame[app_id: int, title: string, content_vec: vector]

In [10]:
# Example
test_game = games.limit(2).collect()[1]
target_app_id = test_game["app_id"]

print("Using first game:")
print("app_id =", test_game["app_id"])
print("title  =", test_game["title"])
print("tags   =", test_game["tags"])


Using first game:
app_id = 302810
title  = Divinia Chronicles: Relics of Gan-Ti
tags   = ['Indie', 'Action RPG', 'Souls-like', 'Retro', 'Soundtrack', 'Third Person', 'Singleplayer', 'Open World', 'Exploration', 'Fantasy', 'Story Rich', 'Adventure', 'RPG', 'Action', 'Early Access', 'Hack and Slash']


In [11]:
target_vec = (
    games_final
        .filter(F.col("app_id") == target_app_id)
        .select("content_vec")
        .collect()[0][0]
)

target_vec_list = target_vec.toArray().tolist()

import numpy as np
from pyspark.sql.functions import udf
from pyspark.ml.linalg import SparseVector

def cosine_sim(a_list, b_vec):
    a = np.array(a_list)
    b = b_vec.toArray() if isinstance(b_vec, SparseVector) else np.array(b_vec)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-9))

cosine_udf = udf(cosine_sim, "double")

similar_games = (
    games_final
        .withColumn("similarity", cosine_udf(F.lit(target_vec_list), "content_vec"))
        .orderBy(F.col("similarity").desc())
        .limit(20)
        .select("app_id", "title", "similarity")
)

similar_games.show(truncate=False)


+-------+-------------------------------------+-------------------+
|app_id |title                                |similarity         |
+-------+-------------------------------------+-------------------+
|302810 |Divinia Chronicles: Relics of Gan-Ti |0.9999999999984093 |
|843620 |Forest Below                         |0.2149898314919096 |
|1032660|Romans from Mars (Free-to-Play)      |0.2104142674356382 |
|1082890|Viking Vengeance                     |0.21025773462690614|
|294680 |Putt-Putt® Enters the Race           |0.20173026371035832|
|447780 |LostWinds                            |0.190671970130919  |
|1363980|时之痕 Trace Of Time                 |0.1895687261141034 |
|907020 |Bomber 95                            |0.1862371701811759 |
|2107270|Closed Island                        |0.18555405516334475|
|539050 |Super Island God VR                  |0.17934832665477543|
|797660 |God Test                             |0.17635208309076547|
|1899980|Monster Island                       |0.17

# Collaborative Filtering

In [12]:
from pyspark.sql import functions as F

ratings = (
    reviews
    .select(
        "user_id", "app_id",
        "is_recommended", "playtime_hours"
    )
)


In [13]:
ratings = ratings.withColumn(
    "score",
    1.0 * F.col("is_recommended").cast("float") +
    0.5 * F.log1p(F.col("playtime_hours"))
)


In [15]:
ratings_sample = (
    ratings.sample(withReplacement=False, fraction=0.05, seed=42)
            .limit(2_000_000)
            .toPandas()
)

In [16]:
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

ratings_sample["user_idx"] = ratings_sample["user_id"].astype("category").cat.codes
ratings_sample["item_idx"] = ratings_sample["app_id"].astype("category").cat.codes

distinct_user_ids = ratings_sample["user_id"].astype("category").cat.categories
distinct_item_ids = ratings_sample["app_id"].astype("category").cat.categories

sparse_matrix = coo_matrix(
    (ratings_sample["score"], (ratings_sample["user_idx"], ratings_sample["item_idx"]))
)

print("Sparse matrix shape:", sparse_matrix.shape)


Sparse matrix shape: (1610470, 27639)


In [17]:
model_knn = NearestNeighbors(metric="cosine", algorithm="brute")
model_knn.fit(sparse_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
item_user_matrix = sparse_matrix.T.tocsr()
print("Item-user matrix:", item_user_matrix.shape)

Item-user matrix: (27639, 1610470)


In [19]:
from sklearn.neighbors import NearestNeighbors

model_knn_items = NearestNeighbors(
    metric="cosine",
    algorithm="brute",
    n_neighbors=50,
    n_jobs=-1
)

model_knn_items.fit(item_user_matrix)


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=50)

In [20]:
def get_items_for_user(target_user_id):
    try:
        user_idx = distinct_user_ids.get_loc(target_user_id)
    except KeyError:
        return []

    row = sparse_matrix.getrow(user_idx)
    item_idxs = row.indices

    return item_idxs

def get_anchor_item(target_user_id):
    item_idxs = get_items_for_user(target_user_id)
    if len(item_idxs) == 0:
        return None
    return item_idxs[0]    # or choose based on highest score

def get_similar_items_for_user(target_user_id, top_k=50):
    anchor_item = get_anchor_item(target_user_id)
    if anchor_item is None:
        return []

    distances, indices = model_knn_items.kneighbors(
        item_user_matrix[anchor_item],
        n_neighbors=top_k+1
    )

    similar_item_idxs = indices.flatten()[1:]   # drop anchor itself
    similar_item_ids = [distinct_item_ids[i] for i in similar_item_idxs]

    return similar_item_ids

def get_cf_df_itembased(target_user_id, top_k=50):
    similar_items = get_similar_items_for_user(target_user_id, top_k=top_k)

    if len(similar_items) == 0:
        return pd.DataFrame(columns=["app_id", "cf_score", "cf_norm"])

    # simple descending scores: 1.0 → 0.0
    cf_scores = np.linspace(1.0, 0.0, len(similar_items))

    cf_df = pd.DataFrame({
        "app_id": similar_items,
        "cf_score": cf_scores
    })

    # normalize (though this is already normalized)
    cf_df["cf_norm"] = (
        (cf_df["cf_score"] - cf_df["cf_score"].min()) /
        (cf_df["cf_score"].max() - cf_df["cf_score"].min() + 1e-9)
    )

    return cf_df


In [21]:
test_user_id = ratings_sample.iloc[0]["user_id"]

cf_df = get_cf_df_itembased(test_user_id, top_k=50)
cf_df.head(10)


,app_id,cf_score,cf_norm
0,394220,1.000000,1.000000
1,773530,0.979592,0.979592
2,1865370,0.959184,0.959184
3,821890,0.938776,0.938776
4,1323420,0.918367,0.918367
5,293180,0.897959,0.897959
6,513780,0.877551,0.877551
7,1827980,0.857143,0.857143
8,513510,0.836735,0.836735
9,368370,0.816327,0.816327


# Hybrid Recommendation System with Reranking

In [22]:
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.linalg import SparseVector


# ----------------------------------
# Helper: cosine similarity for CB
# ----------------------------------
def cosine_sim(a, b):
    a = np.array(a)
    if isinstance(b, SparseVector):
        b = b.toArray()
    else:
        b = np.array(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-9))

In [23]:
def recommend_games(user_id, top_n=20, alpha_cf=0.4, alpha_cb=0.6):
    # --------------------------------------
    # 1. Get user's played games
    # --------------------------------------
    user_played = set(
        ratings_sample[ratings_sample["user_id"] == user_id]["app_id"].tolist()
    )

    print("\nUser's Played Games:")
    if len(user_played) == 0:
        print("  No played games recorded.")
    else:
        spark_user_games = (
            spark.createDataFrame([(int(x),) for x in user_played], ["app_id"])
            .join(games.select("app_id", "title", "tags"), "app_id", "inner")
        )

        for row in spark_user_games.collect():
            print(f"  - {row['title']}")
            print(f"    tags: {row['tags']}")

    # --------------------------------------
    # 2. CF (item-based)
    # --------------------------------------
    cf_df = get_cf_df_itembased(user_id, top_k=150)
    if cf_df is None or cf_df.empty:
        print("No CF results for this user.")
        return None

    # Remove games already played
    cf_df = cf_df[~cf_df["app_id"].isin(user_played)]
    candidate_app_ids = cf_df["app_id"].tolist()

    if len(candidate_app_ids) == 0:
        print("CF has no new games to recommend.")
        return None

    # --------------------------------------
    # 3. CB (content-based)
    # --------------------------------------
    user_games = list(user_played)

    if len(user_games) == 0:
        sample_vec = games_final.limit(1).collect()[0]["content_vec"].toArray()
        user_profile = np.zeros_like(sample_vec)
    else:
        spark_user_games = spark.createDataFrame(
            [(int(x),) for x in user_games], ["app_id"]
        ).join(games_final, "app_id", "inner")

        vecs = [row["content_vec"].toArray() for row in spark_user_games.collect()]
        user_profile = np.mean(np.vstack(vecs), axis=0)

    spark_candidates = spark.createDataFrame(
        [(int(x),) for x in candidate_app_ids], ["app_id"]
    ).join(games_final, "app_id", "inner")

    cb_rows = []
    for row in spark_candidates.collect():
        sim = cosine_sim(user_profile, row["content_vec"])
        cb_rows.append((row["app_id"], sim))

    cb_df = pd.DataFrame(cb_rows, columns=["app_id", "cb_score"])

    # Normalize
    cb_df["cb_norm"] = (
        (cb_df["cb_score"] - cb_df["cb_score"].min()) /
        (cb_df["cb_score"].max() - cb_df["cb_score"].min() + 1e-9)
    )

    # --------------------------------------
    # 4. Hybrid Merge
    # --------------------------------------
    hybrid = (
        pd.merge(cf_df[["app_id", "cf_norm"]],
                 cb_df[["app_id", "cb_norm"]],
                 on="app_id",
                 how="outer")
        .fillna(0)
    )

    hybrid["hybrid_score"] = (
        alpha_cf * hybrid["cf_norm"] +
        alpha_cb * hybrid["cb_norm"]
    )

    # --------------------------------------
    # 5. Attach titles & tags
    # --------------------------------------
    meta = games.select("app_id", "title", "tags").toPandas()
    hybrid = hybrid.merge(meta, on="app_id", how="left")

    # --------------------------------------
    # 6. Filter out games the user already played (final safety)
    # --------------------------------------
    hybrid = hybrid[~hybrid["app_id"].isin(user_played)]

    # --------------------------------------
    # 7. Sort & return
    # --------------------------------------
    hybrid = hybrid.sort_values("hybrid_score", ascending=False)
    final_df = hybrid.head(top_n).reset_index(drop=True)

    print("\nRecommended Games:")
    for idx, row in final_df.iterrows():
        print(f"{idx+1}. {row['title']} (app_id={row['app_id']})")
        print(f"    tags: {row['tags']}")

    return final_df

In [24]:
multi_game_users = (
    ratings_sample.groupby("user_id")["app_id"]
    .nunique()
    .reset_index()
    .sort_values("app_id", ascending=False)
)

multi_game_users.head()

,user_id,app_id
229354,11764552,303
949972,5112758,212
213935,11656130,197
1026182,5669734,173
199646,11553593,160


In [25]:
# Test on a random user with 1 game played
test_user_id = multi_game_users[multi_game_users["app_id"] == 1].iloc[20]["user_id"]
test_user_id

'5998554'

In [26]:
recommend_games(test_user_id, top_n=20)


User's Played Games:
  - Half-Life 2: Genry's Great Escape From City 13
    tags: ['Free to Play', 'Indie', 'Action', 'Adventure', 'FPS', 'Shooter', 'First-Person', 'Military', 'Physics', 'Singleplayer', 'Violent', 'Mod']

Recommended Games:
1. Logistique Act. 1 (app_id=1154130)
    tags: ['Action', 'Adventure', 'Shooter', 'Singleplayer', 'Mod', '3D Platformer', 'Fast-Paced', 'Puzzle', 'FPS']
2. Resistance Element (app_id=1054600)
    tags: ['Action', 'Free to Play', 'Adventure', 'Violent', 'Mod', 'FPS']
3. Half-Life: Caged (app_id=679990)
    tags: ['Free to Play', 'Action', 'Adventure', 'First-Person', 'Mod', 'Shooter', 'Singleplayer', 'Great Soundtrack', 'FPS', 'Sci-fi', '1980s', 'Multiplayer']
4. Half-Life 2: Year Long Alarm (app_id=747250)
    tags: ['Free to Play', 'Action', 'Adventure', 'Violent', 'Mod', 'FPS', 'Singleplayer', 'Horror']
5. Amalgam (app_id=1389950)
    tags: ['Free to Play', 'Mod', 'Action', 'FPS', 'Singleplayer', 'Adventure', 'Shooter', 'First-Person', 'Fightin

,app_id,cf_norm,cb_norm,hybrid_score,title,tags
0,1154130,0.986577,0.734448,0.835300,Logistique Act. 1,"[Action, Adventure, Shooter, Singleplayer, Mod..."
1,1054600,0.993289,0.632397,0.776754,Resistance Element,"[Action, Free to Play, Adventure, Violent, Mod..."
2,679990,0.832215,0.628278,0.709853,Half-Life: Caged,"[Free to Play, Action, Adventure, First-Person..."
3,747250,0.865772,0.581211,0.695036,Half-Life 2: Year Long Alarm,"[Free to Play, Action, Adventure, Violent, Mod..."
4,1389950,0.979866,0.489111,0.685413,Amalgam,"[Free to Play, Mod, Action, FPS, Singleplayer,..."
5,380,0.208054,1.000000,0.683221,Half-Life 2: Episode One,"[FPS, Action, Sci-fi, Singleplayer, First-Pers..."
6,420,0.664430,0.643601,0.651932,Half-Life 2: Episode Two,"[FPS, Action, Sci-fi, Singleplayer, Shooter, S..."
7,340,0.885906,0.473934,0.638723,Half-Life 2: Lost Coast,"[FPS, Action, Singleplayer, First-Person, Sci-..."
8,33110,0.926174,0.215419,0.499721,Alien Shooter: Revisited,"[Action, RPG, Indie, Shooter, Singleplayer, Is..."
9,204340,0.771812,0.302898,0.490463,Serious Sam 2,"[Action, FPS, Co-op, Comedy, Multiplayer, Shoo..."


In [27]:
# Test on a random user with 5 games played
test_user_id = multi_game_users[multi_game_users["app_id"] == 5].iloc[1]["user_id"]
test_user_id


'9650036'

In [28]:
recommend_games(test_user_id, top_n=20)


User's Played Games:
  - BioShock Infinite
    tags: []
  - FTL: Faster Than Light
    tags: ['Roguelike', 'Space', 'Strategy', 'Sci-fi', 'Indie', 'Singleplayer', 'Perma Death', 'Difficult', 'Replay Value', 'Real-Time with Pause', 'Great Soundtrack', '2D', 'Simulation', 'Roguelite', 'Survival', 'Adventure', 'Pixel Graphics', 'RPG', 'Atmospheric', 'Addictive']
  - The White Door
    tags: ['Indie', 'Adventure', 'Puzzle', 'Point & Click', 'Short', 'Detective', 'Mystery', 'Psychological Horror', 'Atmospheric', 'Story Rich', 'Singleplayer', '2D', 'Surreal']
  - Dishonored®: Death of the Outsider™
    tags: ['Stealth', 'Action', 'First-Person', 'Singleplayer', 'Assassin', 'Female Protagonist', 'Steampunk', 'Atmospheric', 'Story Rich', 'Violent', 'Magic', 'Adventure', 'Immersive Sim', 'Choices Matter', 'Dark', 'Open World', 'Gore', 'FPS', 'Parkour', 'Fantasy']
  - BioShock™ 2 Remastered
    tags: ['FPS', 'Action', 'Singleplayer', 'Story Rich', 'Atmospheric', 'Horror', 'Shooter', 'First-Pers

,app_id,cf_norm,cb_norm,hybrid_score,title,tags
0,532110,0.483221,1.000000,0.793289,Rusty Lake: Roots,"[Puzzle, Point & Click, Indie, Adventure, Psyc..."
1,909090,0.651007,0.854587,0.773155,Paradox: A Rusty Lake Film,"[Movie, Short, Psychological Horror, Puzzle]"
2,810630,1.000000,0.615524,0.769314,Paint It Black,"[Action, Indie, Casual, Puzzle, Minimalist, Si..."
3,504690,0.872483,0.626455,0.724866,Woven,"[Adventure, Casual, Indie, Story Rich, Singlep..."
4,31860,0.979866,0.524305,0.706529,Nancy Drew®: Legend of the Crystal Skull,"[Adventure, Point & Click, Puzzle, Funny, Clas..."
5,1113780,0.919463,0.547594,0.696342,Uncharted Tides: Port Royal,"[Adventure, Casual, Hidden Object, Pirates, 2D..."
6,1292940,0.503356,0.756778,0.655409,Cube Escape Collection,"[Puzzle, Point & Click, Mystery, Psychological..."
7,2221940,0.852349,0.521866,0.654059,The Feast,"[Multiple Endings, Dark, Free to Play, Choices..."
8,720470,0.865772,0.508919,0.651660,Caveman Alive,"[Adventure, Indie, Casual, Simulation]"
9,527420,0.825503,0.489495,0.623898,Demon Hunter 3: Revelation,"[Casual, Adventure, Hidden Object, Point & Cli..."


In [29]:
# Use 0.6 cf + 0.4 cb
recommend_games(test_user_id, top_n=20, alpha_cf=0.6, alpha_cb=0.4)


User's Played Games:
  - BioShock Infinite
    tags: []
  - FTL: Faster Than Light
    tags: ['Roguelike', 'Space', 'Strategy', 'Sci-fi', 'Indie', 'Singleplayer', 'Perma Death', 'Difficult', 'Replay Value', 'Real-Time with Pause', 'Great Soundtrack', '2D', 'Simulation', 'Roguelite', 'Survival', 'Adventure', 'Pixel Graphics', 'RPG', 'Atmospheric', 'Addictive']
  - The White Door
    tags: ['Indie', 'Adventure', 'Puzzle', 'Point & Click', 'Short', 'Detective', 'Mystery', 'Psychological Horror', 'Atmospheric', 'Story Rich', 'Singleplayer', '2D', 'Surreal']
  - Dishonored®: Death of the Outsider™
    tags: ['Stealth', 'Action', 'First-Person', 'Singleplayer', 'Assassin', 'Female Protagonist', 'Steampunk', 'Atmospheric', 'Story Rich', 'Violent', 'Magic', 'Adventure', 'Immersive Sim', 'Choices Matter', 'Dark', 'Open World', 'Gore', 'FPS', 'Parkour', 'Fantasy']
  - BioShock™ 2 Remastered
    tags: ['FPS', 'Action', 'Singleplayer', 'Story Rich', 'Atmospheric', 'Horror', 'Shooter', 'First-Pers

,app_id,cf_norm,cb_norm,hybrid_score,title,tags
0,810630,1.000000,0.615524,0.846209,Paint It Black,"[Action, Indie, Casual, Puzzle, Minimalist, Si..."
1,31860,0.979866,0.524305,0.797641,Nancy Drew®: Legend of the Crystal Skull,"[Adventure, Point & Click, Puzzle, Funny, Clas..."
2,504690,0.872483,0.626455,0.774072,Woven,"[Adventure, Casual, Indie, Story Rich, Singlep..."
3,1113780,0.919463,0.547594,0.770715,Uncharted Tides: Port Royal,"[Adventure, Casual, Hidden Object, Pirates, 2D..."
4,909090,0.651007,0.854587,0.732439,Paradox: A Rusty Lake Film,"[Movie, Short, Psychological Horror, Puzzle]"
5,720470,0.865772,0.508919,0.723031,Caveman Alive,"[Adventure, Indie, Casual, Simulation]"
6,589640,0.966443,0.354343,0.721603,Spoids,"[Strategy, Indie, Action, Tower Defense, Sci-f..."
7,2221940,0.852349,0.521866,0.720156,The Feast,"[Multiple Endings, Dark, Free to Play, Choices..."
8,363580,0.959732,0.360367,0.719986,PING 1.5+™,"[Indie, Puzzle]"
9,756500,0.906040,0.408135,0.706878,Noir Chronicles: City of Crime,"[Adventure, Casual, Hidden Object, Detective, ..."
